# 앙상블 학습과 랜덤 포레스트
* 일련의 예측기(앙상블)로 예측을 수집한다.
* 여러개의 결정트리를 쓰는 결정트리 앙상블을 랜덤 포레스트라고 한다.
* 배깅,부스팅,스태킹등 인기있는 앙상블 기법이 존재한다.
---
### 투표 기반 분류기
* 여러개의 모델중 가장 많이 선택된 클래스를 선택하는 것을 직접투표 라고 한다.
  * 다수결로 인한 약한 분류기 여러개가 매우 좋은 성능을 내는 경우가 많다.
* 가장 뛰어난 모델보돠 정확도가 높을 경우가 많다.
* 큰 수의 법칙으로 앞면이 51퍼센트인 동전을 1000번 하면 앞면이 다수가 될 확률은 75퍼센트 이다.
* 모든 분류기가 독립적이고 오차에 상관관계가 없어야하지만 같은 데이터로 훈련시키기 때문에 가정이 맞지 않다.
  * 분류기는 같은 종류의 오차를 만들기 쉽다.
  * 앙상블 방법은 예측기가 가능한 한 서로 독립적일 때 최고의 성능을 발휘한다.
  * 각기 다른 알고리즘으로 학습시키면 앙상블의 정확도가 높아진다.   

    ![image](https://user-images.githubusercontent.com/62787572/131330778-43ca2b10-4373-4dcb-ba16-6e3c606a53bf.png)


In [ ]:
# Moon 데이터셋 에서의 앙상블 평가.
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
x,y=datasets.make_moons(n_samples=1000,noise=0.4)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()
voting_clf=VotingClassifier(
    estimators=[('lr',log_clf),('rf',rnd_clf),('svm',svm_clf)],
    voting='hard'
)
voting_clf.fit(x_train,y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                             

In [ ]:
from sklearn.metrics import accuracy_score
for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
  clf.fit(x_train,y_train)
  y_pred=clf.predict(x_test)
  print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

LogisticRegression 0.815
RandomForestClassifier 0.795
SVC 0.835
VotingClassifier 0.825


### 베깅과 페이스팅
* 중복해서 뽑는 방식을 배깅, 중복을 허용 안하면 페이스팅   
    ![image](https://user-images.githubusercontent.com/62787572/131331075-2f04a4e9-3eaa-44f1-a3fa-cfc45eb62e69.png)   
* 전형적으로 통계적 최빈값을 분류 진행하고 회귀의 경우 평균 계산한다.

In [ ]:
# 사이킷런의 BaggingClassifier를 사용하면 간편한 배깅 기법 사용 가능.
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500,
    max_samples=100,bootstrap=True,n_jobs=-1,
    oob_score=True
)
bag_clf.fit(x_train,y_train)
y_pred=bag_clf.predict(x_test)
print(bag_clf.__class__.__name__,accuracy_score(y_test,y_pred))
bag_clf.oob_score_

BaggingClassifier 0.81


0.84375

![image](https://user-images.githubusercontent.com/62787572/131331365-80ab554b-e6e4-4736-a9ca-741304c20b29.png)   
> 배깅을 사용할 경우 더 일반화가 잘되는 것으로 보인다.

### OOB 평가
* BaggingClassifier는 기본값으로 m개의 샘플을 선택한다. 이는 평균 63% 정도만 샘플링 된다는 것을 의미한다. 나머지 37%를 OOB라고 한다.
* BaggingClassifier의 oob_score=True로 지정하면 훈련이 끝난 후 자동으로 oob평가를 수행한다. 이는 oob_score_변수에 저장되어 있다.

In [ ]:
y_pred=bag_clf.predict(x_test)
accuracy_score(y_test,y_pred)

0.81

### 랜덤 패치와 랜덤 서브스페이스
* 배깅분류기는 특성 샘플링도 지원한다. 무작위로 입력한 특성의 일부분으로 훈련된다.
* 이는 이미지와 같은 고차원 데이터셋에 유용하다.
* 특성과 샘플을 모두 샘플링하는 것을 램덤 패치 방식, 훈련샘플은 모두 사용하고 특성은 샘플링 하는것을 랜덤 서브 스페이스 방식이라 칭한다.
---
### 랜덤 포레스트
* 결정트리를 배깅의 방식으로 많은 트리를 만들어 내 그중 다수결을 직접투표 하는 방식이다.   
> 랜덤포레스트는 노드를 분할할 때 전체 특성중 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 주입한다. 이는 다양한 트리를 만들어 편향을 손해보는 대신 분산을 낮추어 매우 훌륭한 모델을 만들게 된다.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf=RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(x_train,y_train)
y_pred_rf=rnd_clf.predict(x_test)
accuracy_score(y_test,y_pred_rf)

0.81

### 엑스트라 트리
* 트리를 매우 무작위로 하기 위해 후보 특성을 사용해 무작위로 분할하고 최상의 분할을 선택
* 이렇게 극단적으로 무작위하면 익스트림 랜덤 트리(엑스트라트리)라고 한다.
*최적의 임곗값을 찾지 않으니 일반적으로 빠르다!
---
### 특성 중요도
* 랜덤포레스트의 장점은 상대적 중요도 측정이 쉽다.
* 사이킷런은 어떤 특성 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인해 중요도 측정
* 사이킷런은 훈련이 끝나고 중요도의 전체 합이 1이 되도록 정규화한다.

In [ ]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf=RandomForestClassifier(n_estimators=500,n_jobs=-1)
rnd_clf.fit(iris["data"],iris["target"])
for name,score in zip(iris["feature_names"],rnd_clf.feature_importances_):

  print(name,score)

sepal length (cm) 0.10002569605807488
sepal width (cm) 0.022120620182746253
petal length (cm) 0.4473820845002994
petal width (cm) 0.43047159925887946


* Mnist를 랜덤포레스트 분류기로 훈련시키고 중요도를 그래프로 나타내면 아래와 같다.   
![image](https://user-images.githubusercontent.com/62787572/131333513-06568ba3-7a02-4a62-8354-b87bd85a5a42.png)   


### 부스팅
* 약한 학습기를 여러개 연결해서 강한 학습기를 만드는 방식.
* 에이다부스트,그레디언트 부스팅 인기!
___
### 에이다 부스트
* 잘못분류된 샘플의 가중치를 상대적으로 높이는 방식으로 예측을 만든다.
* 경사하강법은 비용함수를 최소화 하는 기법인 반면에 점차 좋아지도록 예측기 추가!
* 모든 예측기가 훈련을 마치면 배깅이나 페이스팅과 비슷하게 예측을 만든다.
* 아래 그림에서 보이듯이 분류에서 과소적합 된 샘플의 중요도를 높여서 분류기를 만드는 방식이다.      
    ![image](https://user-images.githubusercontent.com/62787572/131334121-c9fd8d47-b7f0-4765-b714-eed8d3bad79c.png)
 

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),n_estimators=200,
    algorithm="SAMME.R",learning_rate=0.5
)
ada_clf.fit(x_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          

In [ ]:
ada_clf.score(x_test,y_test)

0.835

### 그레디언트 부스팅
* 이전 학습기의 잔여 오차에 새로운 예측기를 학습시킨다.
* 학습률을 낮게 설정하면 많은 트리가 필요하지만 성능은 좋아진다.(축소)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
np.random.seed(42)
x_train = np.random.rand(100, 1) - 0.5
y_train= 3*x_train[:, 0]**2 + 0.05 * np.random.randn(100)
tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(x_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=2,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
y2=y_train-tree_reg1.predict(x_train)
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(x_train,y2)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=2,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
y3=y2-tree_reg2.predict(x_train)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(x_train,y3)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=2,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
y_pred=sum(tree.predict(x_train) for tree in (tree_reg1,tree_reg2,tree_reg3))
y_pred

array([0.04021166, 0.49995198, 0.17052257, 0.04021166, 0.29044761,
       0.29044761, 0.6679558 , 0.49995198, 0.04021166, 0.17052257,
       0.6679558 , 0.67888336, 0.49995198, 0.29044761, 0.29044761,
       0.29044761, 0.04021166, 0.04021166, 0.04021166, 0.04021166,
       0.04021166, 0.49484029, 0.04021166, 0.04021166, 0.04021166,
       0.17052257, 0.29044761, 0.04021166, 0.04021166, 0.6679558 ,
       0.04021166, 0.29044761, 0.6679558 , 0.49995198, 0.49995198,
       0.17052257, 0.04021166, 0.49484029, 0.04021166, 0.04021166,
       0.49484029, 0.04021166, 0.6679558 , 0.49995198, 0.29044761,
       0.04021166, 0.04021166, 0.04021166, 0.04021166, 0.29044761,
       0.49995198, 0.17052257, 0.49995198, 0.49995198, 0.04021166,
       0.49995198, 0.49484029, 0.29044761, 0.6679558 , 0.04021166,
       0.04021166, 0.29044761, 0.49995198, 0.04021166, 0.29044761,
       0.04021166, 0.29044761, 0.17052257, 0.49484029, 0.75026781,
       0.17052257, 0.29044761, 0.6679558 , 0.17052257, 0.17052

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt=GradientBoostingRegressor(max_depth=2,n_estimators=300,learning_rate=0.1)
gbrt.fit(x_train,y_train)

gbrt_slow = GradientBoostingRegressor(max_depth=2, n_estimators=200, learning_rate=0.1, random_state=42)
gbrt_slow.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=200,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

* 왼쪽은 트리가 충분하지 않아 세밀하지 못한 반면 오른쪽은 트리가 너무 많아 과대적합이 되어있다.
* 이렇듯 최적의 트리를 찾는 것은 GBRT에서 매우 중요하다.   
![image](https://user-images.githubusercontent.com/62787572/131335452-9c287bcd-0cc3-49fb-847a-13564d336b1d.png)

* 최적의 트리 수를 찾기 위해서 조기 종료 기법을 사용한다.
* 120개의 트리로 GBRT 앙상블을 훈련시키고 최적의 트리 수를 찾기 위해 각 훈련 단계에서 검증 오차를 측정한다.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

x_train,x_val,y_train,y_val=train_test_split(x_train,y_train)

gbrt=GradientBoostingRegressor(max_depth=2,n_estimators=120)
gbrt.fit(x_train,y_train)

errors=[mean_squared_error(y_val,y_pred) for y_pred in gbrt.staged_predict(x_val)]

bst_n_estimators=np.argmin(errors)+1

gbrt_best=GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=74,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
min_error = np.min(errors)

In [ ]:
plt.figure(figsize=(10, 4))

plt.subplot(121)
plt.plot(errors, "b.-")
plt.plot([bst_n_estimators, bst_n_estimators], [0, min_error], "k--")
plt.plot([0, 120], [min_error, min_error], "k--")
plt.plot(bst_n_estimators, min_error, "ko")
plt.text(bst_n_estimators, min_error*1.2, "Minimum", ha="center", fontsize=14)
plt.axis([0, 120, 0, 0.01])
plt.xlabel("Number of trees")
plt.ylabel("Error", fontsize=16)
plt.title("Validation error", fontsize=14)

* 조기종료를 사용한 최적의 트리수    
![image](https://user-images.githubusercontent.com/62787572/131335668-b95c3c52-d67b-4a4c-b427-7d1a2b9bf666.png)

* 아래 코드를 사용하면 5회 반복 동안 성능이 향상되지 않으면 훈련을 멈추고 최적의 모델을 사용한다.

In [ ]:
gbrt=GradientBoostingRegressor(max_depth=2,warm_start=True)

min_val_error=float("inf")
error_going_up=0
for n_estimators in range(1,120):
  gbrt.n_estimators=n_estimators
  gbrt.fit(x_train,y_train)
  y_pred=gbrt.predict(x_val)
  val_error=mean_squared_error(y_val,y_pred)
  if val_error < min_val_error:
    min_val_error=val_error
    error_going_up=0
  else:
    error_going_up+=1
    if error_going_up==5:
      break

### XGBoost
* 최적화된 그레디언트 부스팅 구현으로 XGBoost가 매우 성능이 좋다.
---
### 스태킹
* 앙상블에 속한 모든 예측기의 예측을 취합하는 모델을 훈련시키는 방법?!
* 회귀 모델 여러개의 결과값을 취합해서 최종 예측을 만드는 모델!(최종 예측기를 블렌더 또는 메타학습기라 부름)
* 홀드아웃 세트를 이용한다.
  * 훈련세트를 두개의 서브셋으로 나눈다.
    * 첫번째 서브셋은 레이어의 예측을 훈련시키기 위해 사용된다.   
![image](https://user-images.githubusercontent.com/62787572/131336547-4d26e8d1-9b74-4c51-8d36-15fb80958084.png)
    * 첫번째 레이어의 예측기를 사용해 두번째 세트의 예측을 만든다.   
    ![image](https://user-images.githubusercontent.com/62787572/131336674-f2e2a918-97b1-48b9-9298-575c1c07d288.png) 
    * 첫번째 세트의 결과를 취합하는 두번째 세트의 블렌더를 통해 최종 예측을 진행한다.      
![image](https://user-images.githubusercontent.com/62787572/131336290-e4bab2e2-8bfe-4d1c-9771-984e3ead150f.png)
---
# 연습문제
1. 정확히 같은 훈련 데이터로 다섯 개의 다른 모델을 훈련시켜 이 모델을 연결해 더 좋은 결과를 얻을 수 있을까?
> Voting의 방식을 이용해 결과값의 투표를 진행한다면 더 높은 결과를 얻을 수 있다. 또한 독립적으로 훈련되었다면 더 좋은 성능 향상을 보일 것이다.
---
2.직접 투표와 간접 투표 분류기의 차이점은?
> 이는 Soft voting 과 Hard voting의 문제이다. 직접투표는 해당 투표의 결과중 다수결을 따르는 것이고 간접투표는 해당 클래스에 대한 예측 확률의 평균을 통해 투표하는 방식이다.   
---
3. 배깅,페이스팅,부스팅,랜덤포레스트,스테킹의 훈련을 여러 대의 서버에 분산시켜 속도를 높일 수 있을까??   
* 배깅,페이스팅,랜덤포레스트,스태킹은 서로 독립적이라 병렬 연산이 가능하다.
* 부스팅은 이전 예측기의 결과값이 다음에 영향을 미치니 불가능하다.   
---
4. Oob평가의 장점은 무엇인가?
> 예측기가 훈련되는 동안에 OOB샘플은 선택되지 않으므로 별도의 훈련세트가 필요 없이 검증이 가능하다.  
---
5. 무엇이 엑스트라 트리를 랜덤포레스트보다 더 무작위하게 만드는가??
> 최적의 임계값을 찾기보다 더 무작위로 분할하는 세트를 만든다. 이를 통해 편향이 추가되고 분산이 줄어든다. 또한, 최적의 임계값을 찾지 않으니 매우 빠른 속도를 기대할 수 있다.   
---
6. 에이다 부스트 앙상블이 과소적합 되었다면 어떤 매개변수를 사용해야하나??
> 에이다 n_estimator 변수를 사용해 추정기의 수를 늘인다.
---
7. 그레디언트 부스팅 앙상블이 과대적합 되었다면 학습률을 높여야하나??
> 학습률을 높일 경우 더 많은 트리를 필요로 하며 정확도가 올라간다. 따라서 학습률을 낮추어야 한다.
---
8. MNist 데이터로 여러 분류기를 훈련시킨 후 직접 또는 간접 투표를 실시해 테스트 셋에서 검증하라.

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,VotingClassifier
from sklearn.svm import LinearSVC
mnist = fetch_openml('mnist_784')
x_train=mnist.data[:1500]
y_train=mnist.target[:1500]
x_test=mnist.data[1500:1700]
y_test=mnist.target[1500:1700]
rf=RandomForestClassifier(n_estimators=300,max_depth=50)
ec=ExtraTreesClassifier(n_estimators=300,max_depth=30)
ls=LinearSVC()
rf.fit(x_train,y_train)
ec.fit(x_train,y_train)
ls.fit(x_train,y_train)
print(rf.score(x_test,y_test),ec.score(x_test,y_test),ls.score(x_test,y_test))
# hard보팅을 사용한다. linearsvc는 predict-proba를 사용하기 위한 probability 변수가 존재하지 않는다. 따라서 soft보팅을 사용 불가하다.
voting=VotingClassifier(estimators=[ ('lr', rf), ('rf', ec), ('gnb', ls)],voting='hard')
voting.fit(x_train,y_train)
print(voting.score(x_test,y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.895 0.91 0.82
0.905


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


9. 이전 연습문제의 각 분류기를 실행해 예측을 만들고 그 결과로 새로운 훈련세트를 만든다. 후, 새로운 훈련세트로 분류기를 훈련시킨다. 이를 스태킹이라 한다.

In [14]:
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,VotingClassifier
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
mnist = fetch_openml('mnist_784')
x_train=mnist.data[:1500]
y_train=mnist.target[:1500]
x_test=mnist.data[1500:1700]
y_test=mnist.target[1500:1700]
x_val=mnist.data[1700:1800]
y_val=mnist.target[1700:1800]
# 랜덤포레스트, 엑스트라 트리, 선형 SVC 사용
rf=RandomForestClassifier(n_estimators=300,max_depth=50)
ec=ExtraTreesClassifier(n_estimators=300,max_depth=30)
ls=LinearSVC()
rf.fit(x_train,y_train)
ec.fit(x_train,y_train)
ls.fit(x_train,y_train)
# Val데이터에서 예측기의 예측값을 데이터 입력값으로 랜덤포레스트를 훈련시킨다.
df=pd.DataFrame({'rf':rf.predict(x_val).astype(np.float32),'ec':ec.predict(x_val).astype(np.float32),'ls':ls.predict(x_val).astype(np.float32)})
rf2=RandomForestClassifier(n_estimators=300,max_depth=50)
rf2.fit(df,y_val)
# test데이터에서 예측기의 예측값을 데이터 입력값으로 랜덤포레스트의 출력의 성능 확인.
df2=pd.DataFrame({'rf':rf.predict(x_test).astype(np.float32),'ec':ec.predict(x_test).astype(np.float32),'ls':ls.predict(x_test).astype(np.float32)})
rf2.score(df2,y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.915